In [1]:
CONFIG_PATH = "../config.yaml"
from p_drought_indices.analysis.DeepLearning.pipeline_gwnet import data_preparation
dataloader = data_preparation(CONFIG_PATH)

The ERA5 raster has spatial dimensions: (0.10000000419197502, -0.1000000004915847)
NDVI dataset resolution: (0.043782177927351104, 0.04378217792735114)
Precipitation dataset resolution (0.10000000419197502, -0.1000000004915847)
The features have dimensions: (730, 92, 1)
The instance have dimensions: (730, 92, 1)
x shape:  (707, 12, 92, 1) , y shape:  (707, 12, 92, 1)
train x:  (511, 12, 92, 1) y: (511, 12, 92, 1)
val x:  (73, 12, 92, 1) y: (73, 12, 92, 1)
test x:  (146, 12, 92, 1) y: (146, 12, 92, 1)


In [4]:
dataloader["scaler"]

In [6]:
import os 
from p_drought_indices.functions.function_clns import load_config
from p_drought_indices.analysis.DeepLearning.pipeline_gwnet import asym_adj
import pickle

CONFIG_PATH = "../config.yaml"
config = load_config(CONFIG_PATH)
adj_type = 'doubletransition'

adj_path = os.path.join(config["DEFAULT"]["data"], "graph_net/adj_dist.pkl")
#sensor_ids, sensor_id_to_ind, adj_mx = load_adj(adj_path,  adj_type)

In [13]:
adj_mx = load_adj(adj_path,  adj_type)

In [14]:
adj_mx

[matrix([[0.29796177, 0.2630017 , 0.18086573, ..., 0.        , 0.        ,
          0.        ],
         [0.17608954, 0.19949663, 0.1760903 , ..., 0.        , 0.        ,
          0.        ],
         [0.08674923, 0.12614492, 0.1429124 , ..., 0.        , 0.        ,
          0.        ],
         ...,
         [0.        , 0.        , 0.        , ..., 0.09515225, 0.08429149,
          0.05859623],
         [0.        , 0.        , 0.        , ..., 0.10941748, 0.12351567,
          0.10941647],
         [0.        , 0.        , 0.        , ..., 0.11185217, 0.1608994 ,
          0.1816326 ]], dtype=float32),
 matrix([[0.29796177, 0.2630017 , 0.18086573, ..., 0.        , 0.        ,
          0.        ],
         [0.17608954, 0.19949663, 0.1760903 , ..., 0.        , 0.        ,
          0.        ],
         [0.08674923, 0.12614492, 0.1429124 , ..., 0.        , 0.        ,
          0.        ],
         ...,
         [0.        , 0.        , 0.        , ..., 0.09515225, 0.08429149

In [ ]:

import numpy as np
import matplotlib.pyplot as plt



import seaborn as sns

adj_mx = generate_adj_dist(df)

mask = np.zeros_like(adj_mx)
mask[np.triu_indices_from(mask)] = True
ax_labels = df[['lat', 'lon']].round(3).set_index(["lat","lon"]).index.tolist()
sns.heatmap(adj_mx, mask=mask, cmap="YlGnBu", xticklabels=ax_labels, yticklabels=ax_labels)
plt.xticks(ticks=plt.xticks()[0][::2])  # Adjust the step value (here, I used 2) to control the spacing
plt.yticks(ticks=plt.yticks()[0][::2])

In [ ]:
from p_drought_indices.analysis.DeepLearning.pipeline_gwnet import asym_adj, build_model

adj = [asym_adj(adj_mx), asym_adj(np.transpose(adj_mx))]